In [1]:
from __future__ import print_function,division,absolute_import
import numpy as np
np.random.seed(1337)

In [2]:
#Now Let's define the model
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,MaxPool2D,Activation,Flatten,BatchNormalization
from keras.optimizers import Adam,Adadelta,RMSprop
from keras.losses import categorical_crossentropy
from keras import utils

Using TensorFlow backend.


In [3]:
#
feature_layers_with_batch_norm = [
    Conv2D(64,3,
           padding='same',
           input_shape=(1,100,100)),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(64,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
  
    BatchNormalization(),
    
    Conv2D(128,3),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(128,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
    BatchNormalization(),
    

    Conv2D(256,3),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(256,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
    BatchNormalization(),
    
    
    Conv2D(512,3),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(512,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
    BatchNormalization(),
    
    
    Flatten(),
]

feature_layers_normal = [
    Conv2D(64,3,
           padding='same',
           input_shape=(1,100,100)),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(64,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
  
    
    
    Conv2D(128,3),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(128,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
    
    

    Conv2D(256,3),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(256,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
   
    
    
    Conv2D(512,3),
    Activation('elu'),
    Dropout(0.25),
    Conv2D(512,3),
    Activation('elu'),
    MaxPool2D(pool_size=2),
    
    
    
    
    Flatten(),
]

classification_layers = [
    Dense(512),
    Activation('elu'),
    Dropout(0.5),
    
    
    Dense(256),
    Activation('elu'),
    Dropout(0.5),
    
    Dense(10),
    Activation('softmax')
]


In [4]:
dat = np.load('ASL_Train.npz')
trainX,TrainY = dat['arr_0'],dat['arr_1']
trainY = utils.np_utils.to_categorical(TrainY,10)
trainX = trainX/255
trainX = trainX.astype('float32')
trainX = trainX.reshape((trainX.shape[0],1,100,100)).astype('float32')
print(trainX.shape,trainY.shape)

(4800, 1, 100, 100) (4800, 10)


In [5]:
dat = np.load('ASL_Test.npz')
testX,TestY = dat['arr_0'],dat['arr_1']
testY = utils.np_utils.to_categorical(TestY,10)
testX = testX/255
testX = testX.astype('float32')
testX = testX.reshape((testX.shape[0],1,100,100)).astype('float32')
print(testX.shape,testY.shape)

(200, 1, 100, 100) (200, 10)


In [6]:
normal_model = Sequential(feature_layers_normal+classification_layers)
normal_model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])

from keras.callbacks import TensorBoard,ModelCheckpoint
from os.path import isfile
data_aug_weight_file = 'ASL-new-normal-weights_bentchmark.h5'

if (isfile(data_aug_weight_file)):
    model.load_weights(data_aug_weight_file)

checkpoint = ModelCheckpoint(data_aug_weight_file, monitor='acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs-ASL-normal_benchmark', histogram_freq=0,write_graph=True, write_images=True)
callbacks_list=[checkpoint,tensorboard]

normal_model.fit(trainX, trainY, batch_size=24,epochs=50,verbose=1,shuffle=True,validation_data=(testX, testY),callbacks=callbacks_list)

Train on 4800 samples, validate on 200 samples
Epoch 1/50
4800/4800 [==============================] - 51s - loss: 1.9895 - acc: 0.3019 - val_loss: 1.7413 - val_acc: 0.3550
Epoch 2/50
4800/4800 [==============================] - 50s - loss: 0.7942 - acc: 0.7400 - val_loss: 0.6914 - val_acc: 0.7850
Epoch 3/50
4800/4800 [==============================] - 50s - loss: 0.2968 - acc: 0.9040 - val_loss: 0.3380 - val_acc: 0.9100
Epoch 4/50
4800/4800 [==============================] - 50s - loss: 0.1424 - acc: 0.9560 - val_loss: 0.1271 - val_acc: 0.9650
Epoch 5/50
4800/4800 [==============================] - 50s - loss: 0.0923 - acc: 0.9723 - val_loss: 0.3072 - val_acc: 0.9250
Epoch 6/50
4800/4800 [==============================] - 50s - loss: 0.0552 - acc: 0.9823 - val_loss: 0.0844 - val_acc: 0.9800
Epoch 7/50
4800/4800 [==============================] - 51s - loss: 0.0486 - acc: 0.9838 - val_loss: 0.1387 - val_acc: 0.9750
Epoch 8/50
4800/4800 [==============================] - 50s - loss: 0.0

4800/4800 [==============================] - 49s - loss: 0.0019 - acc: 0.9994 - val_loss: 0.1628 - val_acc: 0.9800
Epoch 30/50
4800/4800 [==============================] - 49s - loss: 0.0053 - acc: 0.9988 - val_loss: 0.1082 - val_acc: 0.9850
Epoch 31/50
4800/4800 [==============================] - 49s - loss: 0.0140 - acc: 0.9971 - val_loss: 0.1055 - val_acc: 0.9800
Epoch 32/50
4800/4800 [==============================] - 49s - loss: 0.0015 - acc: 0.9994 - val_loss: 0.1110 - val_acc: 0.9800
Epoch 33/50
4800/4800 [==============================] - 49s - loss: 0.0059 - acc: 0.9988 - val_loss: 0.0649 - val_acc: 0.9850
Epoch 34/50
4800/4800 [==============================] - 50s - loss: 0.0063 - acc: 0.9983 - val_loss: 0.1295 - val_acc: 0.9850
Epoch 35/50
4800/4800 [==============================] - 49s - loss: 0.0075 - acc: 0.9988 - val_loss: 0.1287 - val_acc: 0.9850
Epoch 36/50
4800/4800 [==============================] - 49s - loss: 0.0151 - acc: 0.9958 - val_loss: 0.0018 - val_acc: 1.0

In [8]:
batch_norm_model = Sequential(feature_layers_with_batch_norm+classification_layers)
batch_norm_model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])

from keras.callbacks import TensorBoard,ModelCheckpoint
from os.path import isfile
data_aug_weight_file = 'ASL-new-normal-weights_bentchmark_batchnorm.h5'

if (isfile(data_aug_weight_file)):
    model.load_weights(data_aug_weight_file)

checkpoint = ModelCheckpoint(data_aug_weight_file, monitor='acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='./logs-ASL-normal_benchmark_batchnorm', histogram_freq=0,write_graph=True, write_images=True)
callbacks_list=[checkpoint,tensorboard]

batch_norm_model.fit(trainX, trainY, batch_size=24,epochs=50,verbose=1,shuffle=True,validation_data=(testX, testY),callbacks=callbacks_list)

Train on 4800 samples, validate on 200 samples
Epoch 1/50
4800/4800 [==============================] - 53s - loss: 3.3916 - acc: 0.3233 - val_loss: 3.4675 - val_acc: 0.1000
Epoch 2/50
4800/4800 [==============================] - 52s - loss: 0.4430 - acc: 0.8627 - val_loss: 2.1184 - val_acc: 0.4500
Epoch 3/50
4800/4800 [==============================] - 52s - loss: 0.1556 - acc: 0.9483 - val_loss: 1.3923 - val_acc: 0.6350
Epoch 4/50
4800/4800 [==============================] - 52s - loss: 0.1088 - acc: 0.9683 - val_loss: 0.2511 - val_acc: 0.9250
Epoch 5/50
4800/4800 [==============================] - 52s - loss: 0.0589 - acc: 0.9821 - val_loss: 0.1686 - val_acc: 0.9500
Epoch 6/50
4800/4800 [==============================] - 52s - loss: 0.0396 - acc: 0.9873 - val_loss: 0.0577 - val_acc: 0.9800
Epoch 7/50
4800/4800 [==============================] - 53s - loss: 0.0236 - acc: 0.9923 - val_loss: 0.0431 - val_acc: 0.9900
Epoch 8/50
4800/4800 [==============================] - 52s - loss: 0.0